
## Call Center Scheduling


Install amplpy, pandas and other packages

In [ ]:
!pip install -q amplpy ampltools pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 30.7 MB/s eta 0:00:00


Setup AMPL and select solver model(s)

In [ ]:
# Google Colab & AMPL integration
MODULES, LICENSE_UUID = ["coin", 'gurobi', "cplex", "highs", "gokestrel"], "42fc7eb6-69aa-445d-b655-3ad24d836541"
from amplpy import tools
from ampltools import cloud_platform_name, ampl_notebook, register_magics

# instantiate AMPL object and register magics
if cloud_platform_name() is None:
    ampl = AMPL() # Use local installation of AMPL
else:
    ampl = tools.ampl_notebook(modules=MODULES, license_uuid=LICENSE_UUID, g=globals())

register_magics(ampl_object=ampl)

Licensed to Bundle #6741.7193 expiring 20241231: INFO 645 Prescriptive Analytics, Prof. Paul Brooks, Virginia Commonwealth University.


Setup Google Drive. Data files can then be accessed from MyDrive on Google Drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Import pandas and use it to read data from file.
Define data.

In [ ]:
import pandas as pd

shift_data = pd.read_excel("/content/drive/MyDrive/INFO645/worker_input.xlsx",
                       sheet_name="Days Off",
                       index_col=0)

S = list(shift_data.index)

O = shift_data.transpose().to_dict('list')

demand_data = pd.read_excel("/content/drive/MyDrive/INFO645/worker_input.xlsx",
                       sheet_name="Demand",
                       index_col=0)

D = list(demand_data.index)
demand = dict(zip(D, demand_data.Demand))

print(S)
print(O)
print(D)
print(demand)

['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
{'Monday': ['Saturday', 'Sunday'], 'Tuesday': ['Sunday', 'Monday'], 'Wednesday': ['Monday', 'Tuesday'], 'Thursday': ['Tuesday', 'Wednesday'], 'Friday': ['Wednesday', 'Thursday'], 'Saturday': ['Thursday', 'Friday'], 'Sunday': ['Friday', 'Saturday']}
['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
{'Monday': 17, 'Tuesday': 13, 'Wednesday': 15, 'Thursday': 19, 'Friday': 14, 'Saturday': 16, 'Sunday': 11}



Define model.

In [ ]:
ampl.eval ('''

reset;

set S;
set O{j in S};
set D;
param demand {i in D} ;

var x {j in S} >= 0 integer;

minimize workers_objective : sum {j in S} x[j] ;

subject to day_constraint {i in D}:
   sum {j in S: i not in O[j]} x[j] >= demand[i];

''')

Provide data to the model.

In [ ]:
ampl.set['D'] = D
ampl.set['S'] = S
for j in S:
    ampl.set['O'][j] = O[j]

ampl.param['demand'] = demand

Display problem formulation.

In [ ]:
ampl.eval('''expand;''')

minimize workers_objective:
	x['Monday'] + x['Tuesday'] + x['Wednesday'] + x['Thursday'] + 
	x['Friday'] + x['Saturday'] + x['Sunday'];

subject to day_constraint['Monday']:
	x['Monday'] + x['Thursday'] + x['Friday'] + x['Saturday'] + x['Sunday']
	 >= 17;

subject to day_constraint['Tuesday']:
	x['Monday'] + x['Tuesday'] + x['Friday'] + x['Saturday'] + x['Sunday']
	 >= 13;

subject to day_constraint['Wednesday']:
	x['Monday'] + x['Tuesday'] + x['Wednesday'] + x['Saturday'] + 
	x['Sunday'] >= 15;

subject to day_constraint['Thursday']:
	x['Monday'] + x['Tuesday'] + x['Wednesday'] + x['Thursday'] + 
	x['Sunday'] >= 19;

subject to day_constraint['Friday']:
	x['Monday'] + x['Tuesday'] + x['Wednesday'] + x['Thursday'] + 
	x['Friday'] >= 14;

subject to day_constraint['Saturday']:
	x['Tuesday'] + x['Wednesday'] + x['Thursday'] + x['Friday'] + 
	x['Saturday'] >= 16;

subject to day_constraint['Sunday']:
	x['Wednesday'] + x['Thursday'] + x['Friday'] + x['Saturday'] + 
	x['Sunday'] >= 11;



Set solver and solve.

In [ ]:
ampl.setOption('solver', 'cplex')
ampl.solve()

CPLEX 22.1.1:  - Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
 - CPXPARAM_Simplex_Display                         0
 - CPXPARAM_MIP_Display                             0
 - CPXPARAM_Barrier_Display                         0
CPLEX 22.1.1: optimal solution; objective 23
5 simplex iterations


Print solution and results.

In [ ]:
# Print results
obj = ampl.getObjective('workers_objective')
print("\n")
#ampl.setOption('display_round', 0);
print("Total profit is: ", obj.get().value(), "\n")
ampl.display('x');



Total profit is:  23.0 

x [*] :=
   Friday  0
   Monday  6
 Saturday  4
   Sunday  0
 Thursday  8
  Tuesday  4
Wednesday  1
;



An optimal solution is to have 2 workers start on Monday, 6 on Tuesday, 7 on Thursday, 3 on Saturday, and 5 on Sunday for a total of 23 workers.